In [2]:
#!pip install tqdm opencv-python SoccerNet
#!pip install soccernet-downloader

In [ ]:
# Define the dataset path
sn_path = "/scratch/czz7bf/soccernet_data/SoccerNetData"

# Ensure the directory exists
import os
os.makedirs(sn_path, exist_ok=True)

In [6]:
# Install the SoccerNet package (if not already installed)
#!pip install git+https://github.com/SoccerNet/SoccerNet.git

# Alternatively, if using a specific downloader module:
#!pip install soccernet-downloader

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/SoccerNet/SoccerNet.git to /tmp/pip-req-build-qt43bwc7
  Running command git clone --filter=blob:none --quiet https://github.com/SoccerNet/SoccerNet.git /tmp/pip-req-build-qt43bwc7
  Resolved https://github.com/SoccerNet/SoccerNet.git to commit 650aa54194f4a5e54a83a974dda899087b59f4b9
  Preparing metadata (setup.py) ... done
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement soccernet-downloader (from versions: none)
ERROR: No matching distribution found for soccernet-downloader


In [7]:
from SoccerNet.Downloader import SoccerNetDownloader

# Initialize the downloader with the dataset path
downloader = SoccerNetDownloader(LocalDirectory=sn_path)
print("SoccerNetDownloader initialized.")

SoccerNetDownloader initialized.


In [8]:
# Download the dataset
downloader.downloadGames(
    files=["Labels-v3.json", "Frames-v3.zip"],
    split=["train", "valid", "test"],
    task="frames",
    verbose=True
)
print("Dataset download initiated. This may take some time...")

/scratch/czz7bf/soccernet_data/SoccerNetData/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/Labels-v3.json already exists
/scratch/czz7bf/soccernet_data/SoccerNetData/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/Frames-v3.zip already exists
/scratch/czz7bf/soccernet_data/SoccerNetData/england_epl/2014-2015/2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal/Labels-v3.json already exists
/scratch/czz7bf/soccernet_data/SoccerNetData/england_epl/2014-2015/2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal/Frames-v3.zip already exists
/scratch/czz7bf/soccernet_data/SoccerNetData/england_epl/2014-2015/2015-02-21 - 18-00 Swansea 2 - 1 Manchester United/Labels-v3.json already exists
/scratch/czz7bf/soccernet_data/SoccerNetData/england_epl/2014-2015/2015-02-21 - 18-00 Swansea 2 - 1 Manchester United/Frames-v3.zip already exists
/scratch/czz7bf/soccernet_data/SoccerNetData/england_epl/2014-2015/2015-02-22 - 19-15 Southampton 0 - 2 Liverpool/Labels-v3.json already exis

In [9]:
# Install required libraries if not already installed
!pip install torch torchvision scikit-learn lime shap matplotlib seaborn

# Import libraries
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, top_k_accuracy_score
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import lime
from lime import lime_image
import shap

Defaulting to user installation because normal site-packages is not writeable


In [10]:
import os
import zipfile

def extract_frames(zip_path, output_dir):
    """
    Extract frames from a zip file to the specified output directory.
    """
    os.makedirs(output_dir, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)
    print(f"Extracted frames to: {output_dir}")

# Define the root directory
root_dir = "/scratch/czz7bf/soccernet_data/SoccerNetData"

# Traverse all leagues, seasons, and match folders
for league in os.listdir(root_dir):
    league_path = os.path.join(root_dir, league)
    if os.path.isdir(league_path):
        for season in os.listdir(league_path):
            season_path = os.path.join(league_path, season)
            if os.path.isdir(season_path):
                for game in os.listdir(season_path):
                    game_dir = os.path.join(season_path, game)
                    if os.path.isdir(game_dir):
                        zip_path = os.path.join(game_dir, "Frames-v3.zip")
                        if os.path.exists(zip_path):
                            extract_frames(zip_path, game_dir)
                        else:
                            print(f"Warning: Frames-v3.zip not found in {game_dir}")

Extracted frames to: /scratch/czz7bf/soccernet_data/SoccerNetData/germany_bundesliga/2016-2017/2016-09-23 - 21-30 Dortmund 3 - 1 SC Freiburg
Extracted frames to: /scratch/czz7bf/soccernet_data/SoccerNetData/germany_bundesliga/2016-2017/2017-02-04 - 20-30 Dortmund 1 - 0 RB Leipzig
Extracted frames to: /scratch/czz7bf/soccernet_data/SoccerNetData/germany_bundesliga/2016-2017/2016-10-22 - 16-30 Ingolstadt 3 - 3 Dortmund
Extracted frames to: /scratch/czz7bf/soccernet_data/SoccerNetData/germany_bundesliga/2016-2017/2016-09-20 - 21-00 Wolfsburg 1 - 5 Dortmund
Extracted frames to: /scratch/czz7bf/soccernet_data/SoccerNetData/germany_bundesliga/2016-2017/2016-12-10 - 17-30 FC Koln 1 - 1 Dortmund
Extracted frames to: /scratch/czz7bf/soccernet_data/SoccerNetData/germany_bundesliga/2016-2017/2016-10-29 - 19-30 Dortmund 0 - 0 Schalke
Extracted frames to: /scratch/czz7bf/soccernet_data/SoccerNetData/germany_bundesliga/2016-2017/2016-12-20 - 22-00 Dortmund 1 - 1 FC Augsburg
Extracted frames to: /scr

In [17]:
import json
import pandas as pd

def parse_annotations(annotation_path):
    """
    Parse a single Labels-v3.json file and extract event metadata.
    """
    with open(annotation_path, "r") as f:
        data = json.load(f)

    events = []
    actions = data.get("actions", {})
    for img_name, action_data in actions.items():
        metadata = action_data.get("imageMetadata", {})
        label = metadata.get("label")
        game_time = metadata.get("gameTime")
        local_path = metadata.get("localpath")

        # Skip invalid entries
        if not label or not game_time or not local_path:
            continue

        event_data = {
            "image_file": img_name,
            "label": label,
            "game_time": game_time,
            "local_path": local_path
        }
        events.append(event_data)

    return events

# Collect all annotations
all_events = []

# Traverse all leagues, seasons, and match folders
for league in os.listdir(root_dir):
    league_path = os.path.join(root_dir, league)
    if os.path.isdir(league_path):
        for season in os.listdir(league_path):
            season_path = os.path.join(league_path, season)
            if os.path.isdir(season_path):
                for game in os.listdir(season_path):
                    game_dir = os.path.join(season_path, game)
                    if os.path.isdir(game_dir):
                        annotation_path = os.path.join(game_dir, "Labels-v3.json")
                        if os.path.exists(annotation_path):
                            events = parse_annotations(annotation_path)
                            all_events.extend(events)

# Convert to DataFrame
events_df = pd.DataFrame(all_events)
print(f"Total events parsed: {len(events_df)}")
print(events_df.head())

Total events parsed: 12764
  image_file             label  game_time  \
0      0.png  Shots off target  1 - 02:06   
1      1.png   Shots on target  1 - 04:08   
2      2.png  Shots off target  1 - 15:11   
3      3.png  Ball out of play  1 - 16:16   
4      4.png  Shots off target  1 - 17:39   

                                          local_path  
0  germany_bundesliga/2016-2017/2016-09-23 - 21-3...  
1  germany_bundesliga/2016-2017/2016-09-23 - 21-3...  
2  germany_bundesliga/2016-2017/2016-09-23 - 21-3...  
3  germany_bundesliga/2016-2017/2016-09-23 - 21-3...  
4  germany_bundesliga/2016-2017/2016-09-23 - 21-3...  


In [18]:
from sklearn.preprocessing import LabelEncoder

# Encode Labels as integers
label_encoder = LabelEncoder()
events_df["label_encoded"] = label_encoder.fit_transform(events_df["label"])
train_df["label_encoded"] = label_encoder.fit_transform(train_df["label"])
val_df["label_encoded"] = label_encoder.transform(val_df["label"])

print(events_df.columns)

Index(['image_file', 'label', 'game_time', 'local_path', 'label_encoded'], dtype='object')


In [19]:
from sklearn.model_selection import train_test_split

# Split the DataFrame into training and validation sets
train_df, val_df = train_test_split(events_df, test_size=0.2, random_state=42)

In [20]:
import torch
from PIL import Image

class SoccerEventDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx]["image_file"]
        local_path = self.dataframe.iloc[idx]["local_path"]
        img_path = os.path.join(self.root_dir, local_path, img_name)
        label = self.dataframe.iloc[idx]["label_encoded"]  # Use encoded label

        # Load image
        image = Image.open(img_path).convert("RGB")

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        # Convert label to tensor
        label = torch.tensor(label, dtype=torch.long)

        return image, label

In [21]:
from torchvision import transforms

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 (ResNet input size)
    transforms.ToTensor(),          # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])

# Create datasets
train_dataset = SoccerEventDataset(train_df, root_dir="/scratch/czz7bf/soccernet_data/SoccerNetData", transform=transform)
val_dataset = SoccerEventDataset(val_df, root_dir="/scratch/czz7bf/soccernet_data/SoccerNetData", transform=transform)

In [22]:
from torch.utils.data import DataLoader

# Define batch size
batch_size = 32

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [23]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load pre-trained ResNet-50
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

# Replace the final fully connected layer
num_classes = len(events_df["label"].unique())  # Number of unique event labels
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model loaded and moved to device: {device}")

Model loaded and moved to device: cpu


In [24]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [27]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in train_loader:
            inputs, labels = batch

            # Debugging: Print shapes
            print(f"Inputs type: {type(inputs)}, Labels type: {type(labels)}")
            print(f"Inputs shape: {inputs.shape}, Labels shape: {labels.shape}")

            inputs, labels = inputs.to(device), labels.to(device)

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Training Loss: {running_loss/len(train_loader)}")

        # Validation phase
        model.eval()
        val_loss = 0.0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for batch in val_loader:
                inputs, labels = batch

                # Debugging: Print types and shapes
                print(f"Validation - Inputs type: {type(inputs)}, Labels type: {type(labels)}")
                print(f"Validation - Inputs shape: {inputs.shape}, Labels shape: {labels.shape}")

                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_accuracy = accuracy_score(all_labels, all_preds)
        print(f"Validation Loss: {val_loss/len(val_loader)}, Accuracy: {val_accuracy:.4f}")
        model.train()

           

In [28]:
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

Inputs type: <class 'torch.Tensor'>, Labels type: <class 'torch.Tensor'>
Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32])
Inputs type: <class 'torch.Tensor'>, Labels type: <class 'torch.Tensor'>
Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32])
Inputs type: <class 'torch.Tensor'>, Labels type: <class 'torch.Tensor'>
Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32])
Inputs type: <class 'torch.Tensor'>, Labels type: <class 'torch.Tensor'>
Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32])
Inputs type: <class 'torch.Tensor'>, Labels type: <class 'torch.Tensor'>
Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32])
Inputs type: <class 'torch.Tensor'>, Labels type: <class 'torch.Tensor'>
Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32])
Inputs type: <class 'torch.Tensor'>, Labels type: <class 'torch.Tensor'>
Inputs shape: torch.Size([32, 3, 

In [80]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

In [81]:
!pip install lime shap

Epoch 1, Training Loss: 1.6648478779462303


AttributeError: 'tuple' object has no attribute 'to'

In [ ]:
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt

explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(
    inputs[0].permute(1, 2, 0).cpu().numpy(),
    lambda x: model(torch.tensor(x).permute(0, 3, 1, 2).to(device)).detach().cpu().numpy(),
    top_labels=1,
    hide_color=0,
    num_samples=1000
)

temp, mask = explanation.get_image_and_mask(
    explanation.top_labels[0],
    positive_only=True,
    num_features=5,
    hide_rest=False
)
plt.imshow(mark_boundaries(temp / 255.0, mask))
plt.show()

In [ ]:
!pip install lime shap

In [ ]:
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt

explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(
    inputs[0].permute(1, 2, 0).cpu().numpy(),
    lambda x: model(torch.tensor(x).permute(0, 3, 1, 2).to(device)).detach().cpu().numpy(),
    top_labels=1,
    hide_color=0,
    num_samples=1000
)

temp, mask = explanation.get_image_and_mask(
    explanation.top_labels[0],
    positive_only=True,
    num_features=5,
    hide_rest=False
)
plt.imshow(mark_boundaries(temp / 255.0, mask))
plt.show()

In [ ]:
import shap

# Create a SHAP explainer
background = torch.zeros((1, 3, 224, 224)).to(device)
explainer = shap.GradientExplainer(model, background)

# Explain predictions
shap_values, indices = explainer.shap_values(inputs[:1].to(device), ranked_outputs=1)
shap.image_plot(shap_values, inputs[:1].cpu().numpy())

In [12]:
!pip install --upgrade certifi

Defaulting to user installation because normal site-packages is not writeable


In [13]:
import os
import requests

# Define the URL and output path
url = "https://download.pytorch.org/models/resnet50-0676ba61.pth"
output_path = os.path.expanduser("~/weights/resnet50-0676ba61.pth")

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Download the file with SSL verification disabled
response = requests.get(url, verify=False)
if response.status_code == 200:
    with open(output_path, "wb") as f:
        f.write(response.content)
    print(f"File downloaded successfully to {output_path}")
else:
    print("Failed to download the file.")

/home/czz7bf/.local/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download.pytorch.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


File downloaded successfully to /home/czz7bf/weights/resnet50-0676ba61.pth


In [14]:
import torch
import torchvision.models as models

# Load pre-trained ResNet-50 without automatic download
model = models.resnet50(weights=None)

# Replace the final fully connected layer
num_classes = len(events_df["label"].unique())  # Number of unique event labels
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Load pre-trained weights manually (excluding the fc layer)
weights_path = os.path.expanduser("~/weights/resnet50-0676ba61.pth")
state_dict = torch.load(weights_path)

# Remove the `fc` layer weights from the state_dict
if 'fc.weight' in state_dict:
    del state_dict['fc.weight']
if 'fc.bias' in state_dict:
    del state_dict['fc.bias']

# Load the remaining weights into the model
model.load_state_dict(state_dict, strict=False)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Model loaded successfully.")

Model loaded successfully.


In [15]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

In [16]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Training Loss: {running_loss/len(train_loader)}")

        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        print(f"Validation Loss: {val_loss/len(val_loader)}")

In [17]:
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

FileNotFoundError: Image file not found: SoccerNetData/england_epl/2016-2017/2017-05-13 - 14-30 Manchester City 2 - 1 Leicester/22.png

In [36]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(model, test_loader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average="macro"))
    print("Recall:", recall_score(y_true, y_pred, average="macro"))
    print("F1 Score:", f1_score(y_true, y_pred, average="macro"))

evaluate_model(model, test_loader)

FileNotFoundError: [Errno 2] No such file or directory: 'SoccerNetData/spain_laliga/2016-2017/2016-09-18 - 21-45 Espanyol 0 - 2 Real Madrid/2.png'

In [ ]:
!pip install lime

In [8]:
# mock_loader.py
import torch
import json
import os
from torchvision.transforms import ToTensor
from PIL import Image

def load_labels(json_path):
    with open(json_path, 'r') as f:
        return json.load(f)

def create_blank_image(width=1280, height=720):
    return torch.zeros((3, height, width), dtype=torch.float32)

def parse_game(folder_path):
    label_path = os.path.join(folder_path, "Labels-v3.json")
    labels = load_labels(label_path)
    actions = []

    for action in labels:
        # Create dummy image and attach annotations
        action_frame = {
            "image": create_blank_image(),
            "bboxes": torch.tensor(action.get("bboxes", [])),
            "lines": action.get("lines", []),
            "links": action.get("links", [])
        }
        actions.append([action_frame])
    return actions


In [10]:
!pip install mock_loader

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement mock_loader (from versions: none)
ERROR: No matching distribution found for mock_loader


In [9]:
from mock_loader import parse_game

game_folder = "SoccerNetData/england_epl/2014-2015/2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal"
game_data = parse_game(game_folder)

print("Loaded", len(game_data), "actions")
print("Image shape (dummy):", game_data[0][0]["image"].shape)


ModuleNotFoundError: No module named 'mock_loader'